<a href="https://colab.research.google.com/github/dustinmcintosh/random-walks/blob/master/M_Dimensional_Random_Walks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import ScalarFormatter
import pandas as pd
from scipy import optimize
from scipy.special import gamma as gammafunc
from astropy.stats import jackknife_stats

plt.rcParams.update({'font.size': 16})

# Making an example random walk sketch

In [0]:
fig,ax = plt.subplots(1, 1, figsize = (4, 4))

# Draw 2-D lattice.
x = []
y = []
for i in range(-2, 4):
  for j in range(-1, 5):
    x.append(i)
    y.append(j)
ax.plot(x, y, '.')

# Draw arrows for the random walk.
ax.arrow(0, 0, .7, 0, head_width=0.2, color='k')
ax.arrow(1, 0, 0, 0.7, head_width=0.2, color='k')
ax.arrow(1, 1, 0, 0.7, head_width=0.2, color='k')
ax.arrow(1, 2, 0, 0.7, head_width=0.2, color='k')
ax.arrow(1, 3, .7, 0, head_width=0.2, color='k')
ax.arrow(2, 3, 0, -0.7, head_width=0.2, color='k')
ax.arrow(2, 2, -.7, 0, head_width=0.2, color='k')
ax.arrow(1, 2, -.7, 0, head_width=0.2, color='k')
ax.arrow(0, 2, -.7, 0, head_width=0.2, color='k')
ax.arrow(.08, 2, 0, -0.7, head_width=0.2, color='k')
ax.arrow(-.08, 1, 0, 0.7, head_width=0.2, color='k')

# Double arrow for end-to-end distance.
ax.arrow(0, 0, -1+.15, 2-np.sqrt(3)*.15, head_width=0.2, color='g')
ax.arrow(-1, 2, 1-.15, -2+np.sqrt(3)*.15, head_width=0.2, color='g')

# Highlight the sites visited.
ax.plot([1, 1, 1, 1, 2, 2, 0, 0, -1],
        [0, 1, 2, 3, 3, 2, 2, 1, 2],
        'o', color='darkorange')
ax.text(-.95, .6, '$R$', color='g', size=20)

# Some text to describe what's going on.
ax.text(-1.8, 3.35, '$N=11$,', color='k', size=20)
ax.text(0.1, 3.35, '$M=2$', color='C0', size=20)
ax.text(0.5, -0.7, '$N_{unique} = 9$', color='darkorange', size=20)

ax.set_aspect('equal', 'datalim')
ax.axis('off')

In [0]:
# A second random walk with only 10 steps (I like round numbers).

fig,ax = plt.subplots(1, 1, figsize = (4, 4))

# Draw 2-D lattice.
x = []
y = []
for i in range(-2, 4):
  for j in range(-1, 5):
    x.append(i)
    y.append(j)
ax.plot(x, y, '.')

# Draw arrows for the random walk.
ax.arrow(0, 0, .7, 0, head_width=0.2, color='k')
ax.arrow(1, 0, 0, 0.7, head_width=0.2, color='k')
ax.arrow(1, 1, 0, 0.7, head_width=0.2, color='k')
ax.arrow(1, 1.92, .7, 0, head_width=0.2, color='k')
ax.arrow(2, 2.08, -.7, 0, head_width=0.2, color='k')
ax.arrow(1, 2, -.7, 0, head_width=0.2, color='k')
ax.arrow(.08, 2, 0, -0.7, head_width=0.2, color='k')
ax.arrow(-.08, 1, 0, 0.7, head_width=0.2, color='k')
ax.arrow(0, 2, 0, 0.7, head_width=0.2, color='k')
ax.arrow(0, 3, -0.7, 0, head_width=0.2, color='k')

# Double arrow for end-to-end distance.
ax.arrow(0, 0, (-1+.05)*.93, (3-np.sqrt(3)*.05)*.93, head_width=0.2, color='g')
ax.arrow(-1, 3, (1-.05)*.93, (-3+np.sqrt(3)*.05)*.93, head_width=0.2, color='g')

# Highlight the sites visited.
ax.plot([1, 1, 1,  2, 0, 0, 0, -1],
        [0, 1, 2,  2, 2, 1, 3, 3],
        'o', color='darkorange')

# Some text to describe what's going on.
ax.text(-1, 1.35, '$R$', color='g', size=20)
ax.text(-1.8, 3.35, '$N=10$', color='k', size=20)
ax.text(0.3, 3.35, '$M=2$', color='C0', size=20)
ax.text(0.5, -0.7, '$N_{unique} = 8$', color='darkorange', size=20)

ax.set_aspect('equal', 'datalim')
ax.axis('off')

# Defining a Random Walker

In [0]:
class RandomWalker:
  """ Creates a random walker at the origin in the given number of dimensions.
  
  Args:
    Mdim: int;  Number of dimensions the walker lives in. 

  Attributes:
    location: list of length MDim; Location in MDim space.
    MDim: int; Number of dimensions the walker lives in.
    traveled_locations: set(tuples of length MDim); Set of locations the walker
      has visited.
    steps_taken: int; Number of steps the walker has taken already.
  """
  def __init__(self, MDim):
    self.location = [0 for x in range(MDim)]
    self.MDim = MDim
    self.traveled_locations = set()
    self.steps_taken = 0

    # List of all the possible moves (up, down, left, right, in, out, ...).
    self._udlr = [[0 for x in range(MDim)] for x in range(2*MDim)]
    for i in range(MDim):
      self._udlr[2*i][i] = -1
      self._udlr[2*i+1][i] = 1
  
  def move(self):
    # Choose which dimension to walk in.
    which_dir = self._udlr[np.random.choice(len(self._udlr))]
    
    # Move to the new location.
    for i in range(self.MDim):
      self.location[i] = self.location[i]+which_dir[i]

    # Remember where you've been.
    self.traveled_locations.add(tuple(self.location))

    self.steps_taken +=1

# Walking in many different dimensions

In [0]:
# Looking at stats across many different dimensionalities at once (constant N).

# List of dimensions to check.
MDim_list = [1,2,3,4,10]
# How many steps in each random walk?
Nsteps = 1000
# Number of trials for each dimensionality?
Ntrial = 10000

# Schema of the data that we'll collect.
rw_data = pd.DataFrame(
    columns=['M', 
             'trial', 
             'unique_locations', 
             'distance']).astype(
                 {'M': 'int32',
                  'trial': 'int32',
                  'unique_locations': 'int32',
                  'distance': 'float64'})

for mDim in MDim_list:
  for trial in range(Ntrial):
    # Doug, a bit more adventurous than Bob, will be wandering through many 
    # different worlds of varying dimensionality.
    doug = RandomWalker(mDim)

    # Go, Doug, go!
    for i in range(Nsteps):
      doug.move()

    # Let's document Doug's work.
    this_df = {'M': mDim,
               'trial': trial,
               'unique_locations': len(doug.traveled_locations), 
               'distance': np.linalg.norm(doug.location)}       
    rw_data = pd.concat([rw_data, pd.DataFrame(this_df, index=[i])])
    i+=1

In [0]:
# Let's see what our random walker, Doug above, has been up to (more plotting!).
dim_list = rw_data['M'].unique()

fig,ax = plt.subplots(1, 2, figsize = (15, 5))

for i, M in enumerate(sorted(dim_list)):
  this_data = rw_data[rw_data['M']==M]

  # Create histogram of R.
  labels, counts = np.unique(np.round(this_data['distance'], int(0)),
                                return_counts=True)
  ax[0].bar(labels, counts, align='center', label="%i" %(M), width=1)

  # Create histogram of Nunique.
  labels, counts = np.unique(np.round(this_data['unique_locations'], int(0)),
                                return_counts=True)
  ax[1].bar(labels, counts, align='center', label="%i" % (M), width=1)

ax[0].set_xlabel('$R$')
ax[0].set_ylabel('Frequency')
ax[0].set_title('Distance from Origin; $N = $%i' % (Nsteps))
ax[0].legend(title='Dimensions ($M$)')
ax[1].set_xlabel('$N_{unique}$')
ax[1].set_title('# Unique Locations Visited; $N = $%i' % (Nsteps))

# This code below adds labels if using Nsteps = 1000 and Ntrials=10000.

ax[0].set_ylim(0, 695)

# Label standard deviation of R.
ax[0].arrow(36, 550, 3, 0, head_length=3.8, head_width=18, color='purple')
ax[0].arrow(39, 550, -3, 0, head_length=3.8, head_width=18, color='purple')
ax[0].text(36, 575, '$\sigma_{R}(M=10)$', color='purple', size=16)

# Label average of R.
ax[0].arrow(31.6, 625, 0, -20, head_length=27, head_width=2.3, color='purple')
ax[0].text(12, 635, '$\\langle R\\rangle (M=10)$', color='purple', size=16)

# Label standard deviation of Nunique.
ax[1].arrow(345, 105, -30, 0, head_width=15, color='darkorange')
ax[1].arrow(315, 105, 30, 0, head_width=15, color='darkorange')
ax[1].text(225, 130, '$\sigma_{N_{unique}}(M=2)$', color='darkorange', size=16)

# Label average of Nunique.
ax[1].arrow(675, 205, 0, -20, head_width=15, color='g')
ax[1].text(465, 215, '$\\langle N_{unique}\\rangle (M=3)$', color='g', size=16)

# Checking out one $M$

In [0]:
# How many dimensions shall we walk in?
Mdim = 100 # Recommend to use something < 5...it doesn't get more interesting.

In [0]:
# Gather some data from multiple random walks in Mdim dimensions.

# Stats to gather for each simulation.
rw_data = pd.DataFrame(
    columns=['num_steps', 
             'trial', 
             'unique_locations', 
             'distance']).astype(
                 {'num_steps': 'int32',
                               'trial': 'int32',
                               'unique_locations': 'int32',
                               'distance': 'float64'})

i = 0 # used for indexing

# List of numbers of steps that we want to test.
num_steps_list = [10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000]
# How many trials to for each number of steps?
num_trials = 100

# Run the simulations.
for num_steps in num_steps_list:
  for trial in range(num_trials):
    # Our loyal bob will be our random walker today.
    bob = RandomWalker(Mdim)
    # Take num_steps steps.
    for j in range(num_steps):
      bob.move()

    # Gather some stats from bob.
    this_df = {'num_steps': num_steps,
               'trial': trial,
               'unique_locations': len(bob.traveled_locations), 
               'distance': np.linalg.norm(bob.location)}       
    rw_data = pd.concat([rw_data, pd.DataFrame(this_df, index=[i])])
    i+=1

In [0]:
# Some Aggregate data for plotting.
agg_data = rw_data.groupby('num_steps').agg(
    mean_dist=pd.NamedAgg(column='distance', aggfunc='mean'),
    sem_dist=pd.NamedAgg(column='distance', aggfunc='sem'),
    std_dist=pd.NamedAgg(column='distance', aggfunc='std'),
    mean_unique_locs=pd.NamedAgg(column='unique_locations', aggfunc='mean'),
    sem_unique_locs=pd.NamedAgg(column='unique_locations', aggfunc='sem'),
    std_unique_locs=pd.NamedAgg(column='unique_locations', aggfunc='std')
)

# Fit power laws to metrics.
def power_law(x, x0, power):
  return x0*(x**power)

dist_opt_model_params, dist_opt_model_cov = optimize.curve_fit(
    power_law,
    agg_data.index,
    agg_data['mean_dist'],
    sigma=agg_data['sem_dist']
)
ulocs_opt_model_params, ulocs_opt_model_cov = optimize.curve_fit(
    power_law,
    agg_data.index,
    agg_data['mean_unique_locs'],
    sigma=agg_data['sem_unique_locs']
)

In [0]:
# Let's do some plotting.
fig,ax = plt.subplots(1, 1, figsize = (11, 7))

fidget_factor = .03 # to keep distributions from overlapping.

# Plot R and N_unique distributions.
ax.scatter(rw_data['num_steps']*(1-fidget_factor), rw_data['distance'], alpha=0.3, color='orange',
           label='$R$         ')
ax.scatter(rw_data['num_steps']*(1+fidget_factor), rw_data['unique_locations'], alpha=0.12, color='b', 
           label='$N_{unique}$')

# Plot y=x line
ax.plot(rw_data['num_steps'], rw_data['num_steps'], 'k', label='$y=x$')

# Can also plot 1/(2*sqrt(x)) if you want.
# ax.plot(rw_data['num_steps'], np.sqrt(rw_data['num_steps'])/2, 'k-.', label='$y=\sqrt{x}/2$')

# Hacky plot to add an empty line to legend.
ax.plot([50], [150], 'w.', label=' $_{       }$')

# Plot means plus or minus standard error for each N for both R and N_unique.
ax.errorbar(agg_data.index*(1-fidget_factor), agg_data['mean_dist'], 
            yerr=agg_data['sem_dist'], barsabove=True, ecolor='green', fmt='g.', 
            label='$\\langle R \\rangle \pm$s.e.m.')
ax.errorbar(agg_data.index*(1+fidget_factor), agg_data['mean_unique_locs'], 
            yerr=agg_data['sem_unique_locs'], barsabove=True, ecolor='red', 
            fmt='r.', label='$\\langle N_{unique} \\rangle \pm$s.e.m.        ')

# Plot power-law fits for R and Nunique.
ax.plot(agg_data.index*(1-fidget_factor), 
        power_law(agg_data.index, *dist_opt_model_params), 
        'g--', 
        label='$\\langle R \\rangle \\approx %5.2f*N^{%5.2f}$' % tuple(dist_opt_model_params))
ax.plot(agg_data.index*(1+fidget_factor), 
        power_law(agg_data.index, *ulocs_opt_model_params), 
        'r--', 
        label='$\\langle N_{unique} \\rangle \\approx %5.2f * N^{%5.2f}$' % tuple(ulocs_opt_model_params))

# Sorting the legend by the length of the labels.
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: len(t[0])))
ax.legend(handles, labels, ncol=2)

ax.set_title('%i-D Random Walk Stats' % (Mdim))
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('# of steps $N$')
ax.set_ylim(1, max(num_steps_list)*2)
ax.grid(b=True, which='major')

# Show actual numbers: 100, not 10^2.
for axis in [ax.xaxis, ax.yaxis]:
    formatter = ScalarFormatter()
    formatter.set_scientific(False)
    axis.set_major_formatter(formatter)

In [0]:
# Compute errors on params based on covariance of fits.
print('R_0 =', np.round(dist_opt_model_params[0],2), '+/-',
      np.round(1.96*np.sqrt(np.diag(dist_opt_model_cov)[0]),2))
print('alpha =', np.round(dist_opt_model_params[1],2), '+/-',
      np.round(1.96*np.sqrt(np.diag(dist_opt_model_cov)[1]),2))
print('N_0 =', np.round(ulocs_opt_model_params[0],2), '+/-',
      np.round(1.96*np.sqrt(np.diag(ulocs_opt_model_cov)[0]),2))
print('beta =', np.round(ulocs_opt_model_params[1],2), '+/-',
      np.round(1.96*np.sqrt(np.diag(ulocs_opt_model_cov)[1]),2))

In [0]:
# Let's also look at the width of the distributions as a function of N.

# Schema of the data we're looking for.
std_plot_data = pd.DataFrame(
    columns=['num_steps', 
             'std_dist_estimate', 
             'd_std_dist_estimate', 
             'std_ulocs_estimate', 
             'd_std_ulocs_estimate',]).astype(
                 {'num_steps': 'int32',
                  'std_dist_estimate': 'float64',
                  'd_std_dist_estimate': 'float64',
                  'std_ulocs_estimate': 'float64',
                  'd_std_ulocs_estimate': 'float64'})

i=0

# For each N
for num_steps in rw_data['num_steps'].unique():

  # Get the standard deviation of R/Nunique and stderr by jackknife.
  dist_data = np.array(
      rw_data[rw_data['num_steps'] == num_steps]['distance'])
  std_dist_estimate, _, std_dist_stderr, _ = jackknife_stats(dist_data,
                                                             np.std, 
                                                             0.95)
  ulocs_data = np.array(
      rw_data[rw_data['num_steps'] == num_steps]['unique_locations'])
  std_ulocs_estimate, _, std_ulocs_stderr, _ = jackknife_stats(ulocs_data,
                                                               np.std,
                                                               0.95)
  # Gather this data.
  this_df = {'num_steps': num_steps,
             'std_dist_estimate': std_dist_estimate,
             'd_std_dist_estimate': std_dist_stderr, 
             'std_ulocs_estimate': std_ulocs_estimate,
             'd_std_ulocs_estimate': std_ulocs_stderr} 
  std_plot_data = pd.concat([std_plot_data, pd.DataFrame(this_df, index=[i])])
  i+=1

# Fit power laws to the standard deviations vs N.
std_dist_opt_model_params, std_dist_opt_model_cov = optimize.curve_fit(
    power_law,
    std_plot_data['num_steps'],
    std_plot_data['std_dist_estimate'],
    sigma=std_plot_data['d_std_dist_estimate']
)
std_ulocs_opt_model_params, std_ulocs_opt_model_cov = optimize.curve_fit(
    power_law,
    std_plot_data['num_steps'],
    std_plot_data['std_ulocs_estimate'],
    sigma=std_plot_data['d_std_ulocs_estimate']
)

In [0]:
# More plotting!
fig,ax = plt.subplots(1, 1, figsize = (11, 7))

# Plot y=sqrt(x).
ax.plot(std_plot_data['num_steps'], 
        std_plot_data['num_steps']**0.5, 
        'k', label='$y=\sqrt{x}$')

# Plot the standard devation of R and Nunique with standard errors.
ax.errorbar(std_plot_data['num_steps'],
            std_plot_data['std_dist_estimate'],
            yerr=std_plot_data['d_std_dist_estimate'],
            fmt='g.',
            label='$\\sigma_R \pm$s.e.')
ax.errorbar(std_plot_data['num_steps'],
            std_plot_data['std_ulocs_estimate'],
            yerr=std_plot_data['d_std_ulocs_estimate'], 
            fmt='r.',
            label='$\\sigma_{N_{unique}} \pm$s.e.')

# Plot the power law fits.
ax.plot(std_plot_data['num_steps'], 
        power_law(std_plot_data['num_steps'], *std_dist_opt_model_params), 
        'g--', 
        label='$\\sigma_R \\approx %5.2f * N^{%5.2f}$' % tuple(std_dist_opt_model_params)) #\\sqrt{\\langle R^2 \\rangle - \\langle R \\rangle^2}
ax.plot(std_plot_data['num_steps'], 
        power_law(std_plot_data['num_steps'], *std_ulocs_opt_model_params), 
        'r--', 
        label='$\\sigma_{N_{unique}} \\approx %5.2f * N^{%5.2f}$' % tuple(std_ulocs_opt_model_params))

ax.set_title('%i-D Random Walk Stats' % (Mdim))
ax.set_yscale('log')
ax.set_xscale('log')

# Sort labels by their length.
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: len(t[0])))
ax.legend(handles, labels, ncol=2, loc="upper left")
ax.set_xlabel('# of steps $N$')
ax.grid(b=True, which='major')
ax.set_ylim(.15, 400)

# Show actual numbers: 100, not 10^2.
for axis in [ax.xaxis, ax.yaxis]:
    formatter = ScalarFormatter()
    formatter.set_scientific(False)
    axis.set_major_formatter(formatter)

In [0]:
# Compute errors on params based on covariance of fits.
print('sigma_(R,0) =', np.round(std_dist_opt_model_params[0],2), '+/-', np.round(1.96*np.sqrt(np.diag(std_dist_opt_model_cov)[0]),2))
print('gamma =', np.round(std_dist_opt_model_params[1],2), '+/-', np.round(1.96*np.sqrt(np.diag(std_dist_opt_model_cov)[1]),2))
print('sigma_(N,0) =', np.round(std_ulocs_opt_model_params[0],2), '+/-', np.round(1.96*np.sqrt(np.diag(std_ulocs_opt_model_cov)[0]),2))
print('delta =', np.round(std_ulocs_opt_model_params[1],2), '+/-', np.round(1.96*np.sqrt(np.diag(std_ulocs_opt_model_cov)[1]),2))

In [0]:
# Use this to dump the parameters and their errors to text, if you want.
# print(np.round(dist_opt_model_params[0],6))
# print(np.round(1.96*np.sqrt(np.diag(dist_opt_model_cov)[0]),6))
# print(np.round(dist_opt_model_params[1],6))
# print(np.round(1.96*np.sqrt(np.diag(dist_opt_model_cov)[1]),6))
# print(np.round(ulocs_opt_model_params[0],6))
# print(np.round(1.96*np.sqrt(np.diag(ulocs_opt_model_cov)[0]),6))
# print(np.round(ulocs_opt_model_params[1],6))
# print(np.round(1.96*np.sqrt(np.diag(ulocs_opt_model_cov)[1]),6))
# print(np.round(std_dist_opt_model_params[0],6))
# print(np.round(1.96*np.sqrt(np.diag(std_dist_opt_model_cov)[0]),6))
# print(np.round(std_dist_opt_model_params[1],6))
# print(np.round(1.96*np.sqrt(np.diag(std_dist_opt_model_cov)[1]),6))
# print(np.round(std_ulocs_opt_model_params[0],6))
# print(np.round(1.96*np.sqrt(np.diag(std_ulocs_opt_model_cov)[0]),6))
# print(np.round(std_ulocs_opt_model_params[1],6))
# print(np.round(1.96*np.sqrt(np.diag(std_ulocs_opt_model_cov)[1]),6))

# Plotting exponents and scaling factors vs $M$

In [0]:
# Data from running above code a few times for different dimensions.
M =	np.array([1, 2, 3, 4, 5, 10, 100])
R_0 =	np.array([0.779838, 0.884891, 0.926015, 0.931015, 0.952267, 1.015835, 1.000392])
dR_0 =	np.array([0.026455, 0.018205, 0.020791, 0.019078, 0.013065, 0.058313, 0.012124])
alpha =	np.array([0.504398, 0.501017, 0.499856, 0.501677, 0.499879, 0.493699, 0.499834])
dalpha =	np.array([0.005444, 0.003344, 0.003646, 0.003343, 0.002233, 0.009629, 0.001996])
N_0 =	np.array([1.554528, 0.908777, 0.768997, 0.828247, 0.872859, 0.94784, 0.995271])
dN_0 =	np.array([0.016253, 0.069062, 0.040927, 0.017671, 0.00913, 0.005811, 0.000775])
beta =	np.array([0.504005, 0.871139, 0.983606, 0.997064, 0.998967, 0.999511, 0.999967])
dbeta =	np.array([0.001707, 0.010962, 0.00642, 0.002479, 0.001219, 0.000727, 0.00009])
sig_R0 =	np.array([0.62362, 0.455787, 0.383872, 0.327477, 0.29819, 0.198515, 0.06855])
dsig_R0 =	np.array([0.031561, 0.011821, 0.011588, 0.00838, 0.01274, 0.040051, 0.008982])
gamma =	np.array([0.495842, 0.502651, 0.500444, 0.506182, 0.502744, 0.507921, 0.507137])
dgamma =	np.array([0.008268, 0.00432, 0.004976, 0.004253, 0.006981, 0.033199, 0.019854])
sig_N0 =	np.array([0.442433, 0.232983, 0.303548, 0.343108, 0.314747, 0.171646, 0.06913])
dsig_N0 =	np.array([0.01614, 0.012645, 0.022738, 0.048714, 0.027231, 0.030177, 0.005178])
delta =	np.array([0.509693, 0.776118, 0.649286, 0.553684, 0.53305, 0.537327, 0.496139])
ddelta =	np.array([0.00595, 0.009128, 0.012658, 0.022533, 0.013833, 0.02742, 0.010316])

In [0]:
# And why not some more plotting!  This time of exponents and scaling factors.
fig,ax = plt.subplots(2, 7, figsize = (8, 8))

gs0 = ax[0, 0].get_gridspec()
# Creating 1 big plot for the first 5 slots on top.
for a in ax[0, 0:5]:
    a.remove()
ax0big = fig.add_subplot(gs0[0, 0:5])

# Get rid of axes that shouldn't be there..
ax0big.spines['right'].set_visible(False)
ax[0, 5].spines['left'].set_visible(False)
ax[0, 5].spines['right'].set_visible(False)
ax[0, 6].spines['left'].set_visible(False)
ax0big.yaxis.tick_left()
ax0big.tick_params(labelright=False, labelleft=True)  # don't put tick labels at the top
ax0big.xaxis.set_ticklabels([])
ax[0, 5].tick_params(labelright=False, labelleft=False)
ax[0, 5].xaxis.set_ticklabels([])
ax[0, 6].tick_params(labelright=False, labelleft=False)
ax[0, 6].xaxis.set_ticklabels([])

# Add diagonal lines for the breaks in the plot.
# I stole this code from somewhere, sorry whoever I stole from. :/
d = .015  # how big to make the diagonal lines in axes coordinates
# arguments to pass to plot, just so we don't keep repeating them
kwargs = dict(transform=ax0big.transAxes, color='k', clip_on=False)
ax0big.plot((1-d, 1+d), (1-2*d, 1+2*d), **kwargs)        # top-left diagonal
ax0big.plot((1 - d, 1 + d), (2*-d, 2*d), **kwargs)  # top-right diagonal
kwargs.update(transform=ax[0, 5].transAxes)  
ax[0, 5].plot((-6*d, 6*d), (1 - 2*d, 1 + 2*d), **kwargs)  
ax[0, 5].plot((1 - 6*d, 1 + 6*d), (1 - 2*d, 1 + 2*d), **kwargs)
ax[0, 5].plot((1-6*d, 1+6*d), (-2*d, 2*d), **kwargs)  
ax[0, 5].plot((-6*d, 6*d), (-2*d, 2*d), **kwargs) 
kwargs.update(transform=ax[0, 6].transAxes)  
ax[0, 6].plot((-6*d, 6*d), (1 - 2*d, 1 + 2*d), **kwargs) 
ax[0, 6].plot((-6*d, 6*d), (-2*d, 2*d), **kwargs) 

# Plot exponent data for M = 1 to 5.
ax0big.errorbar(M[:-2]-.03, alpha[:-2], yerr=dalpha[:-2], fmt='o', label = '$\\alpha$')
ax0big.errorbar(M[:-2]-.01, beta[:-2], yerr=dbeta[:-2], fmt='o', label = '$\\beta$')
ax0big.errorbar(M[:-2]+.01, gamma[:-2], yerr=dgamma[:-2], fmt='o', label = '$\\gamma$')
ax0big.errorbar(M[:-2]+.03, delta[:-2], yerr=ddelta[:-2], fmt='o', label = '$\\delta$')
ax0big.plot([1, 6], [0.5, 0.5], '-', color='dodgerblue', label = None)
ax0big.plot([3, 6], [1, 1], '-', color='darkorange', label = None)
ax0big.plot([.9, 1.1], [0.5, 0.5], '-', color='darkorange', markersize=20, linewidth=2.5)
ax0big.set_ylabel('Exponent')
ax0big.set_xlim(0.75, 5.25)
ax0big.set_ylim(0.4, 1.1)
ax0big.set_yticks([0.5,0.75,1])
ax0big.grid(b=True, which='major', axis='y')
ax0big.legend(ncol=2, loc="center right")
ax0big.set_xticks(range(1,6))
ax0big.tick_params(axis='x', direction='in')

# Plot exponent data for M = 10.
ax[0, 5].errorbar(M[-2]-.03, alpha[-2], yerr=dR_0[-2], fmt='o')
ax[0, 5].errorbar(M[-2]-.01, beta[-2], yerr=dbeta[-2], fmt='o')
ax[0, 5].errorbar(M[-2]+.01, gamma[-2], yerr=dgamma[-2], fmt='o')
ax[0, 5].errorbar(M[-2]+.03, delta[-2], yerr=ddelta[-2], fmt='o')
ax[0, 5].plot([9.5, 10.5], [0.5, 0.5], '-', color='dodgerblue', label = None)
ax[0, 5].plot([9.5, 10.5], [1, 1], '-', color='darkorange', label = None)
ax[0, 5].set_xlim(9.5, 10.5)
ax[0, 5].set_ylim(0.4, 1.1)
ax[0, 5].set_yticks([0.5,0.75,1])
ax[0, 5].grid(b=True, which='major', axis='y')
ax[0, 5].set_xticks([10])
ax[0, 5].tick_params(axis='y', width=0)
ax[0, 5].tick_params(axis='x', direction='in')

# Plot exponent data for M = 100.
ax[0,6].errorbar(M[-1]-.03, alpha[-1], yerr=dalpha[-1], fmt='o')
ax[0,6].errorbar(M[-1]-.01, beta[-1], yerr=dbeta[-1], fmt='o')
ax[0,6].errorbar(M[-1]+.01, gamma[-1], yerr=dgamma[-1], fmt='o')
ax[0,6].errorbar(M[-1]+.03, delta[-1], yerr=ddelta[-1], fmt='o')
ax[0, 6].plot([99.5, 100.5], [0.5, 0.5], '-', color='dodgerblue', label = None)
ax[0, 6].plot([99.5, 100.5], [1, 1], '-', color='darkorange', label = None)
ax[0,6].set_xlim(99.5, 100.5)
ax[0,6].set_ylim(0.4, 1.1)
ax[0,6].set_yticks([0.5,0.75,1])
ax[0,6].grid(b=True, which='major', axis='y')
ax[0,6].set_xticks([100])
ax[0,6].tick_params(axis='y', width=0)
ax[0, 6].tick_params(axis='x', direction='in')


# Creating 1 big plot for the first 5 slots on bottom.
gs1 = ax[1, 0].get_gridspec()
for a in ax[1, 0:5]:
    a.remove()
ax1big = fig.add_subplot(gs1[1, 0:5])

# Get rid of axes that shouldn't be there..
ax1big.spines['right'].set_visible(False)
ax[1, 5].spines['left'].set_visible(False)
ax[1, 5].spines['right'].set_visible(False)
ax[1, 6].spines['left'].set_visible(False)
ax1big.yaxis.tick_left()
ax1big.tick_params(labelright=False, labelleft=True)  # don't put tick labels at the top
ax[1, 5].tick_params(labelright=False, labelleft=False)
ax[1, 6].tick_params(labelright=False, labelleft=False)

# Add diagonal lines for the breaks in the plot.
# I stole this code from somewhere, sorry whoever I stole from. :/
kwargs = dict(transform=ax1big.transAxes, color='k', clip_on=False)
ax1big.plot((1-d, 1+d), (1-2*d, 1+2*d), **kwargs)        # top-left diagonal
ax1big.plot((1 - d, 1 + d), (2*-d, 2*d), **kwargs)  # top-right diagonal
kwargs.update(transform=ax[1, 5].transAxes)  
ax[1, 5].plot((-6*d, 6*d), (1 - 2*d, 1 + 2*d), **kwargs)  
ax[1, 5].plot((1 - 6*d, 1 + 6*d), (1 - 2*d, 1 + 2*d), **kwargs)
ax[1, 5].plot((1-6*d, 1+6*d), (-2*d, 2*d), **kwargs)  
ax[1, 5].plot((-6*d, 6*d), (-2*d, 2*d), **kwargs)
kwargs.update(transform=ax[1, 6].transAxes)  
ax[1, 6].plot((-6*d, 6*d), (1 - 2*d, 1 + 2*d), **kwargs) 
ax[1, 6].plot((-6*d, 6*d), (-2*d, 2*d), **kwargs) 

# Plot scaling factor data for M = 1 to 5.
ax1big.errorbar(M[:-2]-.03, R_0[:-2], yerr=dR_0[:-2], fmt='o', label = '$R_0$')
ax1big.errorbar(M[:-2]-.01, N_0[:-2], yerr=dN_0[:-2], fmt='o', label = '$N_0$')
ax1big.errorbar(M[:-2]+.01, sig_R0[:-2], yerr=dsig_R0[:-2], fmt='o', label = '$\\sigma_{R,0}$')
ax1big.errorbar(M[:-2]+.03, sig_N0[:-2], yerr=dsig_N0[:-2], fmt='o', label = '$\\sigma_{N_{unique},0}$')
R0_theory = [np.sqrt(2/x)*gammafunc((x+1)/2)/gammafunc(x/2) for x in M[:-2]**1.1]
ax1big.plot(M[:-2]**1.1-.03, R0_theory, '-.', color='dodgerblue', label = None)
ax1big.plot([.89, 1.09], [np.sqrt(8/np.pi), np.sqrt(8/np.pi)], '-', color='darkorange', markersize=20)
ax1big.plot([2.89, 3.09], [.659, .659], '-', color='darkorange', markersize=20)
ax1big.set_xlim(0.75, 5.25)
ax1big.set_ylim(0, 1.8)
ax1big.set_ylabel('Scaling Factor')
ax1big.legend(ncol=2)
ax1big.grid(b=True, which='major', axis='y')
ax1big.set_xticks(range(1,6))
ax1big.set_yticks([0,0.5,1,1.5])
# Lots of spaces to put label in the middle of all three plots.
ax1big.set_xlabel('                                      Dimensions ($M$)')

# Plot scaling factor data for M = 10.
ax[1, 5].errorbar(M[-2]-.03, R_0[-2], yerr=dR_0[-2], fmt='o', label = '$R_0$')
ax[1, 5].errorbar(M[-2]-.01, N_0[-2], yerr=dN_0[-2], fmt='o', label = '$N_0$')
ax[1, 5].errorbar(M[-2]+.01, sig_R0[-2], yerr=dsig_R0[-2], fmt='o', label = '$\\sigma_{R,0}$')
ax[1, 5].errorbar(M[-2]+.03, sig_N0[-2], yerr=dsig_N0[-2], fmt='o', label = '$\\sigma_{N_{unique},0}$')
R0_theory = [np.sqrt(2/x)*gammafunc((x+1)/2)/gammafunc(x/2) for x in [9.5, 10.5]]
ax[1, 5].plot([9.5, 10.5], R0_theory, '-.', color='dodgerblue', label = None)
ax[1, 5].set_xlim(9.5, 10.5)
ax[1, 5].set_ylim(0, 1.8)
ax[1, 5].grid(b=True, which='major', axis='y')
ax[1, 5].set_xticks([10])
ax[1, 5].set_yticks([0.5,1,1.5])
ax[1, 5].tick_params(axis='y', width=0)

# Plot scaling factor data for M = 100.
ax[1, 6].errorbar(M[-1]-.03, R_0[-1], yerr=dR_0[-1], fmt='o', label = '$R_0$')
ax[1, 6].errorbar(M[-1]-.01, N_0[-1], yerr=dN_0[-1], fmt='o', label = '$N_0$')
ax[1, 6].errorbar(M[-1]+.01, sig_R0[-1], yerr=dsig_R0[-1], fmt='o', label = '$\\sigma_{R,0}$')
ax[1, 6].errorbar(M[-1]+.03, sig_N0[-1], yerr=dsig_N0[-1], fmt='o', label = '$\\sigma_{N_{unique},0}$')
R0_theory = [np.sqrt(2/x)*gammafunc((x+1)/2)/gammafunc(x/2) for x in [99.5, 100.5]]
ax[1, 6].plot([99.5, 100.5], R0_theory, '-.', color='dodgerblue', label = None)
ax[1, 6].set_xlim(99.5, 100.5)
ax[1, 6].set_ylim(0, 1.8)
ax[1, 6].grid(b=True, which='major', axis='y')
ax[1, 6].set_xticks([100])
ax[1, 6].set_yticks([0.5,1,1.5])
ax[1, 6].tick_params(axis='y', width=0)

fig.tight_layout()


In [0]:
# Plotting the standard deviations w.r.t. M

fig,ax = plt.subplots(1, 1, figsize = (11, 7))

# Plot y = 1/sqrt(x)
ax.plot(M, 1/M**0.5, 'k', label='$y=1/\sqrt{x}$')

# Plot the standard deviations with their standard errors.
ax.errorbar(M, sig_N0, yerr=dsig_N0, fmt='.', label='$\\sigma_{N_{unique},0}$')
ax.errorbar(M, sig_R0, yerr=dsig_R0, fmt='.', label='$\\sigma_{R,0}$')

ax.set_yscale('log')
ax.set_xscale('log')
ax.legend()
ax.set_xlabel('$M$')
ax.grid(b=True, which='major')

# Show numbers, 100, not 10^2.
for axis in [ax.xaxis, ax.yaxis]:
    formatter = ScalarFormatter()
    formatter.set_scientific(False)
    axis.set_major_formatter(formatter)